In [1]:
import pandas as pd
import numpy as np
from sklearn import tree
from collections import Counter
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import train_test_split
from tabulate import tabulate

In [62]:
df1 = pd.read_csv("new_train.csv")

In [64]:
df1['rank_rel'] = np.zeros(len(df1))
df1['rank_rel'] = np.where(df1.click_bool == 1, 1, df1.rank_rel)
df1['rank_rel'] = np.where(df1.booking_bool == 1, 5, df1.rank_rel)

In [65]:
len(df1[df1.rank_rel == 5])

138390

In [66]:
df1 = df1.drop(['Unnamed: 0','prop_score'], axis = 1)

In [68]:
sample_srch_id = np.random.choice(df1.srch_id.unique(), size= int(0.1 * len(df1.srch_id.unique())))
df_sample = df1[df1.srch_id.isin(sample_srch_id)]
print len(df_sample)
print df_sample['click_bool'].corr(df1['click_bool'])
print df_sample['booking_bool'].corr(df1['booking_bool'])

471306
0.9999999999999999
0.9999999999999998


In [104]:
X = df_sample

In [105]:
X_train = X.head(2*len(X)/3) 
X_test = X.tail(len(X)/3)

In [112]:
y_test = X.tail(len(X)/3).rank_rel
y_test2 = X.tail(len(X)/3).position

In [107]:
grouped  = X_train[X_train.click_bool == False].groupby('srch_id')
grouped = grouped.apply(lambda x: x.sample(frac=0.3))
grouped = grouped.apply(lambda x: x) 
X_train = X_train.drop(X_train[X_train.click_bool == False].index).append(grouped)
y_train = X_train[['rank_rel']]
y_train2 = X_train[['position']]

In [109]:
X_train = X_train.drop(['position','rank_rel','click_bool','booking_bool'], axis = 1)
X_test = X_test.drop(['position','rank_rel','click_bool','booking_bool'], axis = 1)

In [110]:
gbm = GradientBoostingClassifier()
model = gbm.fit(X_train, y_train)

In [114]:
pred2 = model.predict(X_test)

In [113]:
print gbm.score(X_test, y_test)

0.8968058968058968


In [115]:
print sum(pred2)

48500.0


In [120]:
print len(pred2), len(pred2[pred2 != 0])

157102 12772


In [121]:
gbm = GradientBoostingClassifier(max_depth = 2)

In [49]:
#y_test2 = df_sample[['position']].loc[X_test.index]
#y_train2 = df_sample[['position']].loc[X_train.index]

In [122]:
model2 = gbm.fit(X_train, y_train2)

In [123]:
pred = model2.predict(X_test)

In [124]:
print pred

[13 13 13 ... 31  7 31]


In [125]:
dfFinal = X_test

In [126]:
dfFinal['position'] = pred
dfFinal['rank_rel'] = pred2

In [127]:
dfFinal = dfFinal.sort_values(by=['srch_id'])

In [128]:
dfFinal

,srch_id,prop_country_id,year,month,prop_location_score1,prop_location_score2,diff_hist_price,usd_diff,star_diff,srch_query_affinity_score,...,random_bool,price_usd,total_price,nr_pers,comp2_rate,comp5_rate,comp8_rate,prop_id,position,rank_rel
3303433,221794,99,2013,4,2.30,0.0138,-728.098749,1225.80,5.0,0.0,...,False,1225.80,1225.80,3,0.0,0.0,0.0,133197,13,0.0
3304189,221854,219,2013,4,2.71,0.0066,62.877556,194.36,3.0,0.0,...,False,194.36,194.36,2,0.0,0.0,0.0,138846,31,0.0
3304188,221854,219,2013,4,3.04,0.0817,125.749341,218.03,3.0,0.0,...,False,218.03,218.03,2,0.0,0.0,0.0,130289,13,0.0
3304186,221854,219,2013,4,2.56,0.0483,-39.988749,537.69,5.0,0.0,...,False,537.69,537.69,2,0.0,0.0,0.0,127402,13,0.0
3304185,221854,219,2013,4,3.09,0.0304,109.971251,387.73,4.0,0.0,...,False,387.73,387.73,2,0.0,0.0,0.0,125075,31,0.0
3304184,221854,219,2013,4,3.43,0.0436,64.601714,307.81,3.0,0.0,...,False,307.81,307.81,2,0.0,0.0,0.0,113183,13,0.0
3304183,221854,219,2013,4,2.77,0.1410,39.931251,457.77,4.0,0.0,...,False,457.77,457.77,2,0.0,0.0,0.0,103457,13,0.0
3304182,221854,219,2013,4,3.00,0.0512,-125.818749,623.52,5.0,0.0,...,False,623.52,623.52,2,0.0,0.0,0.0,92950,13,0.0
3304181,221854,219,2013,4,0.00,0.0039,-113.353593,383.78,4.0,0.0,...,False,383.78,383.78,2,0.0,0.0,0.0,63164,21,0.0
3304180,221854,219,2013,4,2.40,0.0088,48.945370,118.39,3.0,0.0,...,False,118.39,118.39,2,0.0,0.0,0.0,61712,7,0.0


In [129]:
df = dfFinal[['srch_id', 'position', 'rank_rel']]



def dcg(i, rel):

    return (2**rel - 1)/(np.log2(i + 1))



#########################################################################################



# CACULATING DCG



sum_dcg = 0

array_dcg = []



for k in range(0, len(df) - 1):

    if k != len(df) - 1:
        
        if(df['srch_id'].iloc[k] < df['srch_id'].iloc[k + 1]):
 

            sum_dcg += dcg(df['position'].iloc[k],df['rank_rel'].iloc[k])

            array_dcg.append(sum_dcg)
            sum_dcg = 0

        else:

            sum_dcg += dcg(df['position'].iloc[k],df['rank_rel'].iloc[k])

    else:

        sum_dcg += dcg(df['position'].iloc[k],df['rank_rel'].iloc[k])

        array_dcg.append(sum_dcg)

        sum_dcg = 0

# CALCULATING IDCG
array_idcg = np.array([])
t_idcg = np.array([])
sum_idcg = 0

for k in range(0, len(df) - 1):
    if k != len(df) - 1:
        if(df['srch_id'].iloc[k] < df['srch_id'].iloc[k + 1]):
            t_idcg = np.append(t_idcg, df['rank_rel'].iloc[k])
            t_idcg = np.sort(t_idcg)[::-1]
            for j in range(0, len(t_idcg)):
                sum_idcg += dcg(j + 1, t_idcg[j])
                if(sum_idcg > 0):
                    print sum_idcg
            array_idcg = np.append(array_idcg, sum_idcg)
            sum_idcg = 0
            t_idcg = []
        else:
            t_idcg = np.append(t_idcg, df['rank_rel'].iloc[k])   
    else:
        t_idcg = np.append(t_idcg, df['rank_rel'].iloc[k])
        t_idcg = np.sort(t_idcg)[::-1]
        for j in range(0, len(t_idcg)):
            sum_idcg += dcg(j + 1, t_idcg[j])
        array_idcg = np.append(array_idcg, sum_idcg)
        
print 'nDGC = ', (np.sum(array_dcg)/np.sum(array_idcg))


31.0
50.55882236071518
50.55882236071518
50.55882236071518
50.55882236071518
50.55882236071518
50.55882236071518
50.55882236071518
50.55882236071518
50.55882236071518
50.55882236071518
50.55882236071518
50.55882236071518
50.55882236071518
50.55882236071518
50.55882236071518
50.55882236071518
50.55882236071518
50.55882236071518
50.55882236071518
50.55882236071518
50.55882236071518
50.55882236071518
50.55882236071518
31.0
31.0
31.0
31.0
31.0
31.0
50.55882236071518
66.05882236071518
79.40979566099037
91.40223268526115
102.44465548560984
102.44465548560984
102.44465548560984
102.44465548560984
102.44465548560984
102.44465548560984
102.44465548560984
102.44465548560984
102.44465548560984
102.44465548560984
102.44465548560984
102.44465548560984
102.44465548560984
102.44465548560984
102.44465548560984
102.44465548560984
102.44465548560984
102.44465548560984
102.44465548560984
102.44465548560984
102.44465548560984
102.44465548560984
102.44465548560984
102.44465548560984
102.44465548560984
1.0


66.05882236071518
66.05882236071518
66.05882236071518
66.05882236071518
66.05882236071518
66.05882236071518
66.05882236071518
66.05882236071518
66.05882236071518
66.05882236071518
66.05882236071518
66.05882236071518
66.05882236071518
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
31.0
50.55882236071518
66.05882236071518
79.40979566099037
79.40979566099037
79.40979566099037
79.40979566099037
79.40979566099037
79.40979566099037
79.40979566099037
79.40979566099037
79.40979566099037
79.40979566099037
79.40979566099037
79.40979566099037
79.40979566099037
79.40979566099037
79.40979566099037
79.40979566099037
79.40979566099037
79.40979566099037
79.40979566099037
79.40979566099037
79.40979566099037
79.40979566099037
79.40979566099037
79.40979566099037
79.40979566099037
79.40979566099037
79.40979566099037
79.40979566099037
79.40979566099037
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0

31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
50.55882236071518
50.55882236071518
50.55882236071518
50.55882236071518
50.55882236071518
50.55882236071518
50.55882236071518
50.55882236071518
50.55882236071518
50.55882236071518
50.55882236071518
50.55882236071518
50.55882236071518
50.55882236071518
31.0
50.55882236071518
50.55882236071518
50.55882236071518
50.55882236071518
50.55882236071518
50.55882236071518
50.55882236071518
50.55882236071518
50.55882236071518
50.55882236071518
50.55882236071518
50.55882236071518
50.55882236071518
50.55882236071518
50.55882236071518
50.55882236071518
50.55882236071518
50.5588223607

50.55882236071518
50.55882236071518
50.55882236071518
50.55882236071518
50.55882236071518
50.55882236071518
50.55882236071518
50.55882236071518
50.55882236071518
50.55882236071518
50.55882236071518
50.55882236071518
50.55882236071518
50.55882236071518
50.55882236071518
50.55882236071518
50.55882236071518
50.55882236071518
50.55882236071518
50.55882236071518
50.55882236071518
50.55882236071518
50.55882236071518
50.55882236071518
31.0
50.55882236071518
50.55882236071518
50.55882236071518
50.55882236071518
50.55882236071518
50.55882236071518
50.55882236071518
50.55882236071518
50.55882236071518
50.55882236071518
50.55882236071518
50.55882236071518
50.55882236071518
50.55882236071518
50.55882236071518
50.55882236071518
50.55882236071518
50.55882236071518
50.55882236071518
50.55882236071518
50.55882236071518
50.55882236071518
50.55882236071518
50.55882236071518
50.55882236071518
50.55882236071518
31.0
50.55882236071518
66.05882236071518
79.40979566099037
79.40979566099037
79.40979566099037


50.55882236071518
50.55882236071518
50.55882236071518
50.55882236071518
50.55882236071518
50.55882236071518
50.55882236071518
50.55882236071518
50.55882236071518
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
31.0
50.55882236071518
66.05882236071518
66.05882236071518
66.05882236071518
66.05882236071518
66.05882236071518
66.05882236071518
66.05882236071518
66.05882236071518
66.05882236071518
66.05882236071518
66.05882236071518
66.05882236071518
1.0
1.6309297535714575
1.6309297535714575
1.6309297535714575
1.6309297535714575
1.6309297535714575
1.6309297535714575
1.6309297535714575
1.6309297535714575
1.6309297535714575
1.6309297535714575
1.6309297535714575
1.6309297535714575
1.6309297535714575
1.6309297535714575
1.6309297535714575
1.6309297535714575
1.6309297535714575
1.6309297535714575
1.6309297535714575
31.0
50.55882236071518
66.05882236071518
66.05882236071518
66.05882236071518
66.05882236071518
66.05882236071518
66.05882236071518
66.05882236071518
66.058822

...

1.0
1.6309297535714575
2.1309297535714578
2.5616063116448506
2.9484591188793923
3.3046663059874146
3.637999639320748
3.953464516106477
3.953464516106477
3.953464516106477
3.953464516106477
3.953464516106477
3.953464516106477
3.953464516106477
3.953464516106477
3.953464516106477
3.953464516106477
3.953464516106477
3.953464516106477
3.953464516106477
3.953464516106477
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
1.0
1.6309297535714575
2.1309297535714578
2.1309297535714578
2.1309297535714578
2.1309297535714578
2.1309297535714578
2.1309297535714578
2.1309297535714578
2.1309297535714578
2.1309297535714578
2.1309297535714578
2.1309297535714578
2.1309297535714578
2.1309297535714578
2.1309297535714578
2.1309297535714578
2.1309297535714578
2.1309297535714578
2.1309297535714578
2.1309297535714578
2.1309297535714578
2.1309297535714578
2.1309297535714578
2.1309297535714578
2.1309297535714578
2.

66.05882236071518
66.05882236071518
66.05882236071518
66.05882236071518
66.05882236071518
66.05882236071518
66.05882236071518
66.05882236071518
66.05882236071518
66.05882236071518
66.05882236071518
66.05882236071518
66.05882236071518
66.05882236071518
66.05882236071518
66.05882236071518
66.05882236071518
66.05882236071518
66.05882236071518
66.05882236071518
66.05882236071518
66.05882236071518
66.05882236071518
66.05882236071518
66.05882236071518
66.05882236071518
66.05882236071518
66.05882236071518
66.05882236071518
66.05882236071518
66.05882236071518
66.05882236071518
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
50.55882236071518
66.05882236071518
66.05882236071518
66.05882236071518
66.05882236071518
66.05882236071518
66.05882236071518
66.05882236071518
1.0
1.6309297535714575
2.1309297535714578
2.5616063116448506
2.9484591188793923
2.9484591188793923
2.9484591188793923
2.

31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
50.55882236071518
66.05882236071518
79.40979566099037
79.40979566099037
79.40979566099037
79.40979566099037
79.40979566099037
79.40979566099037
79.40979566099037
79.40979566099037
79.40979566099037
79.40979566099037
79.40979566099037
79.40979566099037
79.40979566099037
79.40979566099037
79.40979566099037
79.40979566099037
79.40979566099037
79.40979566099037
79.40979566099037
79.40979566099037
79.40979566099037
79.40979566099037
79.40979566099037
79.40979566099037
79.40979566099037
79.40979566099037
1.0
1.6309297535714575
2.1309297535714578
2.1309297535714578
2.1309297535714578
2.1309297535714578
2.1309297535714578
2.1309297535714578
2.1309297535714578
2.1309297535714578
2.1309297535714578
2.1309297535714578
2.1309297535714578
2.1309297535714578
2.1309297535714578
2.1309297535714578
2.1309297535714578
2.1309297535714578
1.0
1.6309297535714575
2.1309

79.40979566099037
79.40979566099037
79.40979566099037
79.40979566099037
79.40979566099037
79.40979566099037
79.40979566099037
79.40979566099037
79.40979566099037
79.40979566099037
79.40979566099037
79.40979566099037
79.40979566099037
79.40979566099037
79.40979566099037
79.40979566099037
79.40979566099037
79.40979566099037
79.40979566099037
79.40979566099037
79.40979566099037
79.40979566099037
79.40979566099037
79.40979566099037
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
31.0
50.55882236071518
50.55882236071518
50.55882236071518
50.55882236071518
50.55882236071518
50.55882236071518
50.55882236071518
50.55882236071518
50.55882236071518
50.55882236071518
50.55882236071518
50.55882236071518
50.55882236071518
50.55882236071518
50.55882236071518
50.55882236071518
50.55882236071518
50.55882236071518
50.55882236071518
50.55882236071518
50.55882236071518
50.55882236071518
50.55882236071518
50.55882236071518
50.5588

1.6309297535714575
1.6309297535714575
1.6309297535714575
1.6309297535714575
1.6309297535714575
31.0
50.55882236071518
50.55882236071518
50.55882236071518
50.55882236071518
50.55882236071518
50.55882236071518
50.55882236071518
50.55882236071518
50.55882236071518
50.55882236071518
50.55882236071518
50.55882236071518
50.55882236071518
50.55882236071518
50.55882236071518
50.55882236071518
50.55882236071518
50.55882236071518
50.55882236071518
50.55882236071518
50.55882236071518
50.55882236071518
50.55882236071518
50.55882236071518
50.55882236071518
50.55882236071518
50.55882236071518
50.55882236071518
50.55882236071518
50.55882236071518
50.55882236071518
50.55882236071518
50.55882236071518
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
31.0
1.0
1.6309297535714575
1.6309297535714575
1.6309297535714575
1.6309297535714575
1.6309297535714575
1.6309297535714575
1.6309297535714575
1.6309297535714575
